In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



/Users/marijnkoolen/Code/Huygens/republic-project


In [2]:
import pickle

lbd_model_file = "../../data/models/line_break_detector-18th_century.pcl"

with open(lbd_model_file, 'rb') as fh:
    lbd = pickle.load(fh)

In [4]:
import glob

from republic.elastic.republic_elasticsearch import initialize_es
from republic.helper.paragraph_helper import LineBreakDetector
from republic.helper.paragraph_helper import LineReader
import republic.helper.paragraph_helper as para_helper
from republic.helper.text_helper import SkipgramSimilarity

rep_es = initialize_es(host_type='external', timeout=60)


In [26]:

page_line_files = glob.glob("../../data/pages/page_lines-*-*.tsv.gz")
page_line_files


line_reader = LineReader(page_line_files)

for line in line_reader:
    print(line)
    break



reading lines from file ../../data/pages/page_lines-3760-3771.tsv.gz
{'page_id': 'NL-HaNA_1.01.02_3760_0671-page-1340', 'text': 'Ontfangen een Missivé van den Raedt van'}


In [4]:
line = "zelfs plaatse als Secretaris van gemelde Stad"
line = "van de Provincie van Hollandt eude West-Vries-"
para_helper.get_line_words(line)

['van', 'de', 'Provincie', 'van', 'Hollandt', 'eude', 'West-Vries-']

In [5]:

lbd = LineBreakDetector()

In [6]:
line_reader = LineReader(page_line_files)

lbd.set_counters(line_reader)


Step 1: setting unigram counters
reading lines from file ../../data/pages/page_lines-3760-3771.tsv.gz
reading lines from file ../../data/pages/page_lines-3820-3864.tsv.gz
reading lines from file ../../data/pages/page_lines-3772-3804.tsv.gz
reading lines from file ../../data/pages/page_lines-3805-3819.tsv.gz
10474363 lines processed	all:   756920 types	71226602 tokens
Step 2: setting bigram counter
reading lines from file ../../data/pages/page_lines-3760-3771.tsv.gz
reading lines from file ../../data/pages/page_lines-3820-3864.tsv.gz
reading lines from file ../../data/pages/page_lines-3772-3804.tsv.gz
reading lines from file ../../data/pages/page_lines-3805-3819.tsv.gz
10474363 lines processed	all: 1910172 bigrams
Step 3: setting common merge counters
reading lines from file ../../data/pages/page_lines-3760-3771.tsv.gz
reading lines from file ../../data/pages/page_lines-3820-3864.tsv.gz
reading lines from file ../../data/pages/page_lines-3772-3804.tsv.gz
reading lines from file ../../da

In [3]:
lbd.print_counter_stats()

number of lines: 10474363
number of words per line: 6.800089131911888
all:            756920 types	  71226602 tokens
start:          265194 types	  10474363 tokens
mid:            493624 types	  50342776 tokens
end:            227425 types	  10474363 tokens
mid bigrams:   1910172 types	  29860123 tokens
Number of typical merge line ends: 2655
Number of typical merge line starts: 2279
Number of common merge line ends: 78
Number of common merge line starts: 158


In [8]:
# List the most frequent line end words

for word, e_freq in lbd.end_freq.most_common(20):
    m_freq = lbd.mid_freq[word] if word in lbd.mid_freq else 0
    s_freq = lbd.start_freq[word] if word in lbd.start_freq else 0
    print(f'{word: <25}{e_freq: >8}{s_freq: >8}{m_freq: >8}')

.                         1019297    1586  501975
,                          687483    1033 5183286
van                        353798  370326 2828652
de                         269861  322478 2152357
ge-                        201061      46      33
en                         180748  161049 1374021
den                        159798  298440 1198203
te                         143302  114030 1169242
het                        113323   96030  825700
ver-                        82943      12      10
in                          81430   69803  753855
dat                         80346   85380  760148
haar                        78997   60179  470501
voor-                       75282      11      44
Hoogh                       62937   57944  294690
Heere                       62136   12332  110696
aan                         55863   48844  425044
ende                        54798   46779  340398
Requeste                    53496   17139  108015
op                          52671  101897  552875


## Finding Typical Start and End Line Words

Some words almost exclusively appear as the first or last word of a line. Some of these are special cases, like 'ONtfangen', which always occurs at the start of a resolution base on a received Missive. In that case, the word is not a continuation of the word broken of on the previous line. But first words that are often continuations can be detected by looking at which broken off words they continue and how often their combination occurs. 

The same applies to last words of a line. The word 'gevallen' often ends a resolution that has no decision, so is not continued on the next line. 

Using their distinctive statistical characteristics, we sorted typical start and end line words into:
- t**ypical continuation words**: words that always go together with the last word on the previous line to form an existing word. These words should be considered for merging.
- **non-continuation words**: exceptions like 'ONtfangen' that start a paragraph, that should not be considered for merging.
- **typical broken off words**: words that always go together with the first word on the next line to form an existing word. These words should be considered for merging.
- **non-broken off words**: exceptions like 'gevallen' that end a paragraph, that should not be considered for merging.


In [9]:
lbd.typical_start_merged_with["len-"]

Counter({('geval-', 'gevallen'): 113,
         ('oordee-', 'oordeelen'): 2,
         ('Principa-', 'Principalen'): 3,
         ('geval', 'gevallen'): 1,
         ('stel-', 'stellen'): 3,
         ('geyal-', 'geyallen'): 1,
         ('zey-', 'zeylen'): 1,
         ('al-', 'allen'): 1,
         ('verval-', 'vervallen'): 1,
         ('betaa-', 'betaalen'): 6,
         ('sul-', 'sullen'): 1,
         ('Moo-', 'Moolen'): 1,
         ('uitstel-', 'uitstellen'): 1,
         ('voorval-', 'voorvallen'): 1,
         ('val-', 'vallen'): 2,
         ('inval-', 'invallen'): 1,
         ('Mo-', 'Molen'): 1,
         ('voorgeval-', 'voorgevallen'): 1,
         ('Meubi-', 'Meubilen'): 1})

In [10]:
first_word = "len-"
for last_word, merged_word in lbd.typical_start_merged_with[first_word]:
    print(f"{first_word}\t{lbd.mid_freq[first_word]}\t{last_word: <15}\t{lbd.mid_freq[last_word]}\t{lbd.typical_start_merged_with[first_word][(last_word, merged_word)]}\t{merged_word: <20}\t{lbd.mid_freq[merged_word]}")
    

len-	0	geval-         	0	113	gevallen            	88832
len-	0	oordee-        	0	2	oordeelen           	9993
len-	0	Principa-      	0	3	Principalen         	4175
len-	0	geval          	8853	1	gevallen            	88832
len-	0	stel-          	0	3	stellen             	23338
len-	0	geyal-         	0	1	geyallen            	25
len-	0	zey-           	0	1	zeylen              	880
len-	0	al-            	12	1	allen               	5849
len-	0	verval-        	0	1	vervallen           	2762
len-	0	betaa-         	0	6	betaalen            	8379
len-	0	sul-           	2	1	sullen              	55424
len-	0	Moo-           	0	1	Moolen              	242
len-	0	uitstel-       	0	1	uitstellen          	107
len-	0	voorval-       	0	1	voorvallen          	499
len-	0	val-           	0	2	vallen              	3216
len-	0	inval-         	0	1	invallen            	333
len-	0	Mo-            	3	1	Molen               	157
len-	0	voorgeval-     	0	1	voorgevallen        	1393
len-	0	Meubi-         	0	1	Meubilen         

In [11]:
lbd.typical_end_merged_with["tijdt-"]

Counter({('versuym', 'tijdtversuym'): 6, ('verlies', 'tijdtverlies'): 1})

In [12]:
last_word = "tijdt-"
for first_word, merged_word in lbd.typical_end_merged_with[last_word]:
    print(f"{last_word}\t{lbd.mid_freq[last_word]: <15}\t{first_word}\t{lbd.mid_freq[first_word]}\t{lbd.typical_end_merged_with[last_word][(first_word, merged_word)]}\t{merged_word: <20}\t{lbd.mid_freq[merged_word]}")
    

tijdt-	0              	versuym	280	6	tijdtversuym        	54
tijdt-	0              	verlies	1065	1	tijdtverlies        	14


In [13]:
list(lbd.typical_merge_starts)[:10]

['dikant',
 'tricht',
 'litaire',
 'staen',
 'mineeren',
 'thorisatie',
 'soecken',
 'to',
 'res',
 'tannien']

In [14]:
list(lbd.typical_non_merge_starts)[:10]

['Eindelyk',
 '1732',
 'Koomen',
 'Tjassens',
 '150',
 'ONtifangen',
 'Gockinga',
 'Ouwerkerck',
 'DONtfangen',
 'Onder']

In [15]:
first_word = "teerde"
for last_word, merged_word in lbd.typical_start_merged_with[first_word]:
    print(f"{first_word}\t{lbd.mid_freq[first_word]}\t{last_word: <20}\t{lbd.mid_freq[last_word]}\t{lbd.typical_start_merged_with[first_word][(last_word, merged_word)]}\t{merged_word: <20}\t{lbd.mid_freq[merged_word]}")
    

teerde	28	gepermit-           	0	395	gepermitteerde      	3018
teerde	28	Gedepu-             	1	2127	Gedeputeerde        	20881
teerde	28	ver-                	10	2	verteerde           	13
teerde	28	gerappor-           	0	107	gerapporteerde      	1859
teerde	28	Gecommit-           	0	390	Gecommitteerde      	2818
teerde	28	geëxecu-            	0	3	geëxecuteerde       	32
teerde	28	gepermit            	27	1	gepermitteerde      	3018
teerde	28	gepresen-           	0	28	gepresenteerde      	212
teerde	28	gecontrac-          	0	9	gecontracteerde     	33
teerde	28	gedecre-            	0	6	gedecreteerde       	46
teerde	28	presen-             	0	8	presenteerde        	77
teerde	28	compe-              	0	66	competeerde         	407
teerde	28	geaccep-            	0	20	geaccepteerde       	139
teerde	28	difficul-           	0	16	difficulteerde      	55
teerde	28	spec-               	0	3	specteerde          	25
teerde	28	permit-             	0	5	permitteerde        	86
teerde	28	ongepermit-      

## Finding other common line break words

Words like 'de' often are the continuation of a line break word.

In [16]:
lbd.common_start_merged_with['had']

Counter({('ge-', 'gehad'): 659,
         (':', 'had'): 1,
         ('ge', 'gehad'): 2,
         ('voor-', 'voorhad'): 1,
         ('in', 'inhad'): 2,
         ('inge-', 'ingehad'): 3})

In [17]:
lbd.common_start_merged_with['dus']

Counter({('Arnol-', 'Arnoldus'): 36,
         ('Al-', 'Aldus'): 19,
         ('Everhar-', 'Everhardus'): 2,
         ('al-', 'aldus'): 38,
         ('Ferdinan-', 'Ferdinandus'): 15,
         ('Everar-', 'Everardus'): 7,
         ('preten-', 'pretendus'): 2,
         ('Rappar-', 'Rappardus'): 2,
         ('Godefri-', 'Godefridus'): 18,
         ('per-', 'perdus'): 1,
         ('Sigismun-', 'Sigismundus'): 3,
         ('Syno-', 'Synodus'): 7,
         ('Leonar-', 'Leonardus'): 13,
         ('Gerar-', 'Gerardus'): 16,
         ('Bernar-', 'Bernardus'): 37,
         ('Gerhar-', 'Gerhardus'): 3,
         ('Zelan-', 'Zelandus'): 3,
         ('Elar-', 'Elardus'): 1,
         ('Gudwal-', 'Gudwaldus'): 2,
         ('Godesri-', 'Godesridus'): 1,
         ('Edmun-', 'Edmundus'): 2,
         ('Wynan-', 'Wynandus'): 2,
         ('Richar-', 'Richardus'): 6,
         ('Conrar-', 'Conrardus'): 1,
         ('Meynar-', 'Meynardus'): 1,
         ('Wigbol-', 'Wigboldus'): 2,
         ('repan-', 'repandus'

In [19]:
import pickle

lbd_model_file = "../../data/models/line_break_detector-18th_century.pcl"

with open(lbd_model_file, 'wb') as fh:
    pickle.dump(lbd, fh)

## An algorithm for deciding to merge or not

This notebook was used to develop the line break detection algorithm, by iteratively parsing lines and determining whether there is a line break linking a line to its successor. As rules are added that decide merge or not, the algorithm outputs the line end and line start words and potential merge. That way, rules are eyeballed for mistakes.

To avoid developing the rules for a specific subset of the lines (e.g. because the lines are fed in the same order, starting with the same inventory each time), the line reader should randomise the order of the inventory files.

In [149]:
import glob
import re
import random

data_dir = "../../data/pages"

data_files = glob.glob(f"{data_dir}/page_lines-*.tsv.gz")
# remove any files that contain multiple inventory numbers
data_files = [data_file for data_file in data_files if re.search("page_lines-\d{4}.tsv.gz", data_file)]
# sort in order of inventory number
data_files = sorted(data_files)
# map data files to dictionary with inventory number as key
line_file_map = {}
for data_file in data_files:
    inv_num = int(data_file.split('page_lines-')[-1].replace('.tsv.gz', ''))
    line_file_map[inv_num] = data_file


In [29]:
from republic.helper.metadata_helper import doc_id_to_iiif_url
prev_line = None
for li, line in enumerate(line_reader):
    if prev_line and prev_line["text"].endswith('--'):
        print(li-1, prev_line["text"], '\t', prev_line["page_id"])
        print(doc_id_to_iiif_url(prev_line["page_id"]))
        print(li, line["text"])
        print(para_helper.get_line_words(prev_line["text"]))
        print()
    prev_line = line
    if (li+1) % 50000 == 0:
        break

reading lines from file ../../data/pages/page_lines-3760-3771.tsv.gz
6791 sal werden geconsenteert, sullende het selve be-- 	 NL-HaNA_1.01.02_3760_0130-page-259
https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0130.jpg/2400,0,2500,3500/full/0/default.jpg
6792 staen uyt acht kloecke Schepen van Oorlogh,
['sal', 'werden', 'geconsenteert', ',', 'sullende', 'het', 'selve', 'be-']

8052 Hoogh Mogende hem als dan met eenen or-- 	 NL-HaNA_1.01.02_3760_0644-page-1287
https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0644.jpg/2400,0,2500,3500/full/0/default.jpg
8053 donneerden het Commando van het Guarni-
['Hoogh', 'Mogende', 'hem', 'als', 'dan', 'met', 'eenen', 'or-']

11434 mogen werden. Waer op gedelibereerdt zijn-- 	 NL-HaNA_1.01.02_3760_0264-page-527
https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0264.jpg/2400,0,2500,3500/full/0/default.jpg
11435 de, is goedtgevonden ende verstaen, dat de
['mogen', 'werden', '.'

In [197]:
lbd.common_non_merge_starts

{'&',
 '15',
 '18',
 '25',
 'Acte',
 'Admiraliteit',
 'Advis',
 'Amsterdam',
 'April',
 'Articul',
 'Berlyn',
 'Bout',
 'Breda',
 'Brieven',
 'C',
 'Capiteyn',
 'Collegie',
 'Collonel',
 'Commissarissen',
 'Commissie',
 'Committé',
 'Compagnie',
 'Compagnien',
 'Comptoir',
 'Consul',
 'DD',
 'DE',
 'EE',
 'Edele',
 'Extract',
 'Fagel',
 'Gelderland',
 'Generaal',
 'Goederen',
 'Gouverneur',
 'Graave',
 'Grave',
 'Groot',
 'Groot-Brittannien',
 'Haer',
 'Hage',
 'Ham',
 'Heer',
 'Heere',
 'Heim',
 'Hertogh',
 'Hof',
 'Holland',
 'Hollandt',
 'Hoog',
 'Hoorn',
 'IS',
 'Jan',
 'Johan',
 'July',
 'Koning',
 'Koningh',
 'Koopman',
 'Lande',
 'Landen',
 'Lands',
 'Landt',
 'Lynden',
 'Maaze',
 'Major',
 'Mandament',
 'Memorie',
 'Minister',
 'Mog',
 'Ontfanger',
 'Orange',
 'Ordonnantie',
 'Pasporten',
 'Penningen',
 'Persoonen',
 'Petitie',
 'Plaatsen',
 'Placaat',
 'Prince',
 'Provincien',
 'Puissances',
 'Raad',
 'Raaden',
 'Raadt',
 'Raedt',
 'Rapport',
 'Regiment',
 'Register',
 'Renswo

In [243]:
from republic.helper.metadata_helper import doc_id_to_iiif_url

common_terms = [term for term in lbd.mid_freq if lbd.mid_freq[term] > 100]
print(len(common_terms))
skip_sim = SkipgramSimilarity(ngram_length=2, terms=common_terms)

page_line_files = []
for _ in range(30):
    inv_num = random.randint(3760, 3864)
    if line_file_map[inv_num] not in page_line_files:
        page_line_files.append(line_file_map[inv_num])
        
page_line_files

line_reader = LineReader(page_line_files)


prev_words = []
match_count = 0
prev_line = None
no_merge_count = 0
merge_count = 0
unknown = 0

for li, curr_line in enumerate(line_reader):
    curr_words = para_helper.get_line_words(curr_line["text"])
    #print(curr_words)
    terms = get_line_words(curr_line["text"])
    do_merge, use_merge = para_helper.determine_line_break(lbd, curr_words, prev_words, skip_sim)
    if do_merge is True:
        merge_count += 1
    elif do_merge is False:
        no_merge_count += 1
    else:
        unknown += 1
    if (li+1) % 10000 == 0:
        print(li+1, merge_count, 'merges', no_merge_count, 'non-merges', unknown, 'undecided')
    prev_words = curr_words
    prev_line = curr_line
    if (li+1) % 500000 == 0:
        break
    continue


13393
reading lines from file ../../data/pages/page_lines-3846.tsv.gz
hehoonlyke            0       1	decentie             19     113     144	hehoonlykedecentie       0
1e                   65     259	embruilleeren         1       0       1	1eembruilleeren       0
uittersten          244      20	misnoegt              4      10      14	uitterstenmisnoegt       0
et                  141      33	gedeeltelyk          58     409     513	etgedeeltelyk         0
1650                 39      10	gesloten             87     600     786	1650gesloten          0
jamais              482      73	permettre            29     129     171	jamaispermettre       0
chetchera             0       1	toujours             96     549     736	chetcheratoujours       0
sien-                 0      70	genasseert            1       3       5	siengenasseert        0
Vrugten             175      20	onbruikbaar           9      50      66	Vrugtenonbruikbaar       0
sterk               336      70	toenamen              7

indirect            453      65	belarg                1       1       4	indirectbelarg        0
Mcg                  15      14	sia                  11      20      35	Mcgsia                0
soodamge              3       1	maalr                 1       0       1	soodamgemaalr         0
gesogte              13       3	quaestien            13      70      93	gesogtequaestien       0
deff                  1       1	sterken              17      56      82	deffsterken           0
preetensie-           0       4	maakten              22     103     137	preetensiemaakten       0
uitsluitende          1       1	bevoegtheid          12     106     131	uitsluitendebevoegtheid       0
wordem                8      69	toegelegd            43     257     361	wordemtoegelegd       0
ongewoon             35       6	teeken               87     303     446	ongewoonteeken        0
anderë                0       4	gezoute              15      56      78	anderëgezoute         0
titull                0      

malkander           113      13	grensende            29     113     153	malkandergrensende       0
Betig-                0       2	tse                   4       7      15	Betigtse              0
hoogstdeselven-       0       1	personeelyk           6      10      18	hoogstdeselvenpersoneelyk       0
Otto                694      96	willem               11      11      89	Ottowillem            0
schreeuwende         25       8	beswaaren            63     465     620	schreeuwendebeswaaren       0
voordeele           657      97	ezpresselyk           1       1       2	voordeeleezpresselyk       0
lde-                  0       7	selvel                2       5       7	ldeselvel             0
weesendlyke-          0       1	reproches             5      62      75	weesendlykereproches       0
peremptoire         135      16	faiten               30     220     272	peremptoirefaiten       0
confessie           111      12	brengt               32      48     105	confessiebrengt       0
volgendet

reis                449      64	geschikte            26     116     164	reisgeschikte         0
Cabbottagie           0       1	vaarende            136     516     732	Cabbottagievaarende       0
tevens              718     115	wisten               75     486     623	tevenswisten          0
Kameren             730      66	commoniceeren         1       0       2	Kamerencommoniceeren       0
Confes-               0       5	digne                31      46      91	Confesdigne           0
den-                 40     587	147                   9       5      15	den147                0
besloote            206      27	hegtenis              8      57      74	beslootehegtenis       0
engagement          350      59	hidde                 6      19      27	engagementhidde       0
sentiment           560      96	pretenselyk          28     157     207	sentimentpretenselyk       0
Russen               84      16	geanimeert            8      74      92	Russengeanimeert       0
bestencig-            0 

onderneemingen      205      34	aanleiding           80     402     534	onderneemingenaanleiding       0
bez                   6      16	denken              158     290     509	bezdenken             0
soortgelyke         598      72	onaangenaamheeden      24      78     117	soortgelykeonaangenaamheeden       0
afte                 11       4	danken               61     123     214	aftedanken            0
Weeskameren         200     202	see                  12      45      97	Weeskamerensee        0
verschillende       277      45	begrippen            19     122     156	verschillendebegrippen       0
driemaal            235      34	tierschouten         31       1      32	driemaaltierschouten       0
geloof-               0      21	waardigheid          13      40      64	geloofwaardigheid       0
gedeponeerde         15       6	ietwes               56     407     508	gedeponeerdeietwes       0
bevreest            177      33	gem-ikt               1       0       1	bevreestgem-ikt       0

...

Savat                17       1	desonaengesien        1       5       7	Savatdesonaengesien       0
gesustineerde       407      68	diesaengaende         4      46      57	gesustineerdediesaengaende       0
ladite              457      44	demande              79     563     739	laditedemande         0
decidé               39       6	quels                86     296     420	decidéquels           0
pretensions         193      24	fondées               8      57      73	pretensionsfondées       0
plaintes            427      57	souvent              16     143     182	plaintessouvent       0
peuvent             371      39	porter               87     345     470	peuventporter         0
drost                15       1	commun               20     143     187	drostcommun           0
desdits             179      30	comptes               7      48      64	desditscomptes        0
ans                 283      47	consecutifs           1       8       9	ansconsecutifs        0
pire                  

hare-                 1       3	domesticque          17      97     123	haredomesticque       0
geweldighlijck        6       2	overgevoert          11      52      76	geweldighlijckovergevoert       0
remarquable          12       4	actien               57     241     333	remarquableactien       0
450000 146708 merges 301079 non-merges 2213 undecided
gyselinge           216      24	ontslaen             56     433     560	gyselingeontslaen       0
Wateren             329      44	gaven               160     734     989	Waterengaven          0
wederzydts          431      58	ingenomene            5      36      49	wederzydtsingenomene       0
afzijn               13       1	bedacht              17     141     182	afzijnbedacht         0
toereyckent          22       5	antwoort             98     694     866	toereyckentantwoort       0
geledene            132      25	schaden              88     533     691	geledeneschaden       0
alrede               93       7	jonger               18    

Amies                16       4	desirent              3      11      15	Amiesdesirent         0
expedier            131      25	ulterieurement        4      25      36	expedierulterieurement       0
Passepurt             0       1	necessaire           58     362     488	Passepurtnecessaire       0
Tonneau               8       1	rempli                6      28      35	Tonneaurempli         0
gehecle              58      13	queestieuse           6      24      36	geheclequeestieuse       0
huine                 5       5	betoonde             17      92     137	huinebetoonde         0
Gouve-                0       1	e-                   11      21     653	Gouvee-               0
intormee-             0       1	rem                  34      35      80	intormeerem           0
Week-                17      12	blad                  4       8      14	Weekblad              0
zullende-             0      48	opgekogt             14      74     106	zullendeopgekogt       0
preetext             73  

geem                  1      57	mindere              74     418     563	geemmindere           0
etait                 8       2	amical                2       1       3	etaitamical           0
croyait               0       1	pouvoir             117     610     823	croyaitpouvoir        0
Proclamatien        137      14	voortgang            69     300     427	Proclamatienvoortgang       0
nillioen             28       1	vyfmaal              27     152     214	nillioenvyfmaal       0
strafwaardige        10       5	misdaad              42     208     298	strafwaardigemisdaad       0
Noorder-             11      96	avons                18     164     208	Noorderavons          0
possage               0       1	dlune                 3       4       9	possagedlune          0
présenter            32       2	ci-joint             11      82     116	présenterci-joint       0
llanken               0       1	waarschynelyk        15     105     131	llankenwaarschynelyk       0
Capitulation        10

Doore                 0       1	logtige               2       0       2	Doorelogtige          0
Eede                383      65	gelterkt              1       0       1	Eedegelterkt          0
onbegrypelyk         20       4	voorkomt            129     538     752	onbegrypelykvoorkomt       0
minst               249      38	kostbaar             46     205     271	minstkostbaar         0
Werf                165      34	vorderden            11      42      63	Werfvorderden         0
490000 158389 merges 329104 non-merges 2507 undecided
abuis                88      17	veroorzaakt          44     221     321	abuisveroorzaakt       0
Kerkenraad          761      94	telkens             106     655     867	Kerkenraadtelkens       0
Vaandrig             86      28	4l5ergeanten          1       0       1	Vaandrig4l5ergeanten       0
277                   2       5	113                  23      12      39	277113                0
113                  12      23	36                  159     295     5

## Common contractions

The common merge ends and starts reveal typical contractions of common words like:
- `Requestevan`
- `Missivevan`

In [218]:
lbd.common_end_merged_with['Requeste']

Counter({('van', 'Requestevan'): 45232,
         ('-', 'Requeste'): 2,
         ('l', 'Requestel'): 1})

In [390]:
mid_bigram_freq[('Missive', 'van')]

157693

## Testing the line break detector on paragraph generation from sessions

The 5th resolution (17th paragraph) of the 1705-02-20 resolutions has the following lines:

<pre>
Op de Requeste van den Collonel de la
Cost de Crose, onder het Regiment Dragon-
ders van Walef, is naer voorgaende deliberatie
goedtgevonden ende verstaen, dat een Pasport-
in behoorlijcke forma sal werden gedepescheert,
om vyftien Paerden naer Sluys in Vlaende-
ren te mogen uytvoeren, vry, ende sonder
betalinge van 's Lands gerechtigheyt, mits dat het
jaer, hayr en teken van de voorschreve Paerden
op het eerste Comptoir op het Pasport sal moeten
werden aengetekent.
</pre>

A previous version of line break detection led to the following paragraph:

<pre>
Op de Requeste van den Collonel de la Cost de Crose, onder het Regiment Dragonders van Walef, is naer voorgaende deliberatie goedtgevonden ende verstaen, dat een Pasportin behoorlijcke forma sal werden gedepescheert, om vyftien Paerden naer Sluys in Vlaenderen te mogen uytvoeren, vry, ende sonder betalinge van 's Lands gerechtigheyt, mits dat het jaer, hayr en teken van de voorschreve Paerden op het eerste Comptoir op het Pasport sal moeten werden aengetekent.
</pre>

The word `Pasport` is stuck to the next word `in`.

The new version leads to this:

<pre>
Op de Requeste van den Collonel de la Cost de Crose, onder het Regiment Dragonders van Walef, is naer voorgaende deliberatie goedtgevonden ende verstaen, dat een Pasport - in behoorlijcke forma sal werden gedepescheert, om vyftien Paerden naer Sluys in Vlaenderen te mogen uytvoeren, vry, ende sonder betalinge van 's Lands gerechtigheyt, mits dat het jaer, hayr en teken van de voorschreve Paerden op het eerste Comptoir op het Pasport sal moeten werden aengetekent.
</pre>

In [68]:
session_date = "1705-02-20"
session = rep_es.retrieve_session_by_date(session_date, session_index="session_lines")
session.id

'session-1705-02-20-num-1'

In [53]:
#compare_paragraphs(session)

session-1705-02-20-num-1-para-1
session-1705-02-20-num-1-para-2
session-1705-02-20-num-1-para-3
session-1705-02-20-num-1-para-4
session-1705-02-20-num-1-para-5
session-1705-02-20-num-1-para-6
---------------
DIFFERENCE
---------------
Heemskerck , de Lange, van Hoey, Raedt-
---------------
Heemskerck , de Lange, van Hoey, Raedt
---------------



session-1705-02-20-num-1-para-7
session-1705-02-20-num-1-para-8
---------------
DIFFERENCE
---------------
Wellant, Renswoude, met een Extraordi-
---------------
Wellant, Renswoude, met een Extraordi
---------------



session-1705-02-20-num-1-para-9
session-1705-02-20-num-1-para-10
session-1705-02-20-num-1-para-11
session-1705-02-20-num-1-para-12
session-1705-02-20-num-1-para-13
---------------
DIFFERENCE
---------------
Ontfangen een Missive van den Heere van Haersolte, haer Hoogh Mogende extraordinaris Envoyé aen de Hoven van de Koningen van Sweeden ende Poolen, geschreven tot Danzicht den rr. deser loopende maendt, houdende advertentie. Wa